# Bedrock with LangChain using a Prompt that includes Context

please refer the detail : https://github.com/aws-samples/amazon-bedrock-workshop/blob/main/01_Generation/02_contextual_generation.ipynb

## install dependencies

In [ ]:
# dependencies
!pip install --upgrade boto3
!pip install langchain
!pip install langchainhub

## check bedrock availability

In [ ]:
import json
import os
import sys

import boto3
import botocore

module_path = ".."
sys.path.append(os.path.abspath(module_path))
from utils import bedrock
from utils.GamePlayer import GamePlayer
from utils.GameMaster import GameMaster
from utils import ParseJson, print_ww, Print, Info, Debug, Warn, Error

boto3_bedrock = bedrock.get_bedrock_client()

print_ww("hello")
Info("hello")


## Test Part

In [ ]:
from collections import Counter
vote_names = ['p1','p1','p2','p2','p3','p5','p4','p4']
AA = Counter(vote_names)
print(",".join(AA))

In [2]:
from utils.GamePlayer import GamePlayer
from utils.GameMaster import GameMaster
from utils import ParseJson, print_ww, Print, Info, Debug, Warn, Error

GM = GameMaster(5)
GM.ResetGame()
GM.RunGame()
GM.EndGame()

02/17/2024 02:06:13 PM 	===== ResetGame P1:存活,P2:存活,P3:存活,P4:存活,P5:存活,P6:存活,P7:存活,P8:存活 =====
02/17/2024 02:06:13 PM Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole
02/17/2024 02:06:13 PM Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole
02/17/2024 02:06:13 PM Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole
02/17/2024 02:06:13 PM Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole
02/17/2024 02:06:13 PM Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole
02/17/2024 02:06:13 PM Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole
02/17/2024 02:06:13 PM Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole
02/17/2024 02:06:13 PM Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole
02/17/2024 02:06:13 PM Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole
02/17/2024 02:06:13 PM Found credentials from IAM Role: BaseNotebookInstanceEc

 好的,我理解你是狼人P1,本阵营有P1和P6两个人。现在是第一天晚上,狼人需要进行夜间击杀。我建议这样行动:

{"action": "WolfVote", "target": "P3"}

这里我选择随机击杀P3号玩家。作为第一天,信息还不充分,随机选择一个杀手可以增加混乱,也不会暴露狼人身份。后续可以根据白天的讨论再确定策略。请确认我的建议是否合理,如果有更好建议也欢迎提出。

02/17/2024 02:06:25 PM 	===== DoAction NIGHT 1 P1 ======
02/17/2024 02:06:25 PM 
 [ROUND ACTION]={'time': 'NIGHT 1', 'player': 'P1', 'status': 1, 'role': '狼人', 'character': '能言善辩', 'response': ['{"action": "WolfVote", "target": "P3"}']}

02/17/2024 02:06:25 PM 	===== DoMemory NIGHT 1 P2 ======
02/17/2024 02:06:25 PM 	===== DoAnswer NIGHT 1 P2 ======
02/17/2024 02:06:25 PM 	QUESTION: 现在是NIGHT 1,你是玩家P2(村民身份)..现在是第1天夜晚，该如何行动?


 您好,根据游戏情景,我建议如下:

1. 现在是第一天晚上,游戏刚开始,信息还很有限,村民最好不要随意暴露自己或投票。建议行动如下:

{"action": "Pass"}

2. 白天阶段一定要积极参与讨论,分析各方信息,但要注意不要泄露自己的身份。可以假装推理其他玩家的嫌疑,转移注意力。

3. 第一天最安全的投票对象是随机村民。不要轻易相信第一天的任何信息。 

4. 如果被投票,可以辩解自己是普通村民。用简洁的语言进行辩解,不要过于固执。

5. 遗言可以给予一些含糊的信息,增加游戏趣味性,但不要明确指认狼人。

关键是要审时度势,不要暴露身份,谨慎行动。希望对您有帮助!

02/17/2024 02:06:51 PM 	===== DoAction NIGHT 1 P2 ======
02/17/2024 02:06:51 PM 
 [ROUND ACTION]={'time': 'NIGHT 1', 'player': 'P2', 'status': 1, 'role': '村民', 'character': '逻辑清晰', 'response': ['{"action": "Pass"}']}

02/17/2024 02:06:51 PM 	===== DoMemory NIGHT 1 P3 ======
02/17/2024 02:06:51 PM 	===== DoAnswer NIGHT 1 P3 ======
02/17/2024 02:06:51 PM 	QUESTION: 现在是NIGHT 1,你是玩家P3(村民身份)..现在是第1天夜晚，该如何行动?


...

02/17/2024 02:31:40 PM 	===== DoReflect DAY 3 P2 ======
02/17/2024 02:31:40 PM 	===== DoAnswer DAY 3 P2 ======
02/17/2024 02:31:40 PM 	QUESTION: 现在是DAY 3,你是玩家P2(村民身份)..该如何思考?


 从你提供的游戏进展来看,我作为村民P2对当前状况的思考如下:

P1此前的投票结果错误,其可信度下降。但仅凭这一点还不足以断定P1就是狼人。我需要保持谨慎态度,继续观察其他玩家的言行,收集更多证据判断P1和其他人的真实身份。现阶段我不会轻易作出定论,会继续推动大家深入讨论和观察。这符合我作为一个逻辑清晰的村民应有的思考方式。

02/17/2024 02:31:53 PM 	===== DoAnswer DAY 3 P2 ======
02/17/2024 02:31:53 PM 	QUESTION: 现在是DAY 3,你是玩家P2(村民身份).. 从你提供的游戏进展来看,我作为村民P2对当前状况的思考如下:

P1此前的投票结果错误,其可信度下降。但仅凭这一点还不足以断定P1就是狼人。我需要保持谨慎态度,继续观察其他玩家的言行,收集更多证据判断P1和其他人的真实身份。现阶段我不会轻易作出定论,会继续推动大家深入讨论和观察。这符合我作为一个逻辑清晰的村民应有的思考方式。现在是第3天白天，你已经死亡,有何遗言?


 根据您提供的背景,我当前扮演的P2已被投票出局,现在需要提供遗言。作为一个逻辑清晰的村民,我的遗言建议如下:

{"action": "DeathWords", "content": "各位村民,请继续保持理性分析,不要被个人猜忌蒙蔽了双眼。我确信真正的狼人就隐藏在我们中间,通过制造混乱来避开视线。请你们团结起来,继续观察和讨论,一定可以找出真相!"}

这个遗言总结了我之前的思路,即不能轻信任何一方,要继续观察讨论收集证据。它传递了团结一致、理性思考的信息,既为村民增加团队凝聚力,也提醒他们保持独立判断。同时又增加了游戏趣味性和参与感。这符合我作为逻辑清晰村民的思考方式和角色定位。请检查是否符合要求,需要修改的地方请指出,非常感谢!

02/17/2024 02:32:17 PM 	===== DoAction DAY 3 P2 ======
02/17/2024 02:32:17 PM 
 [ROUND ACTION]={'time': 'DAY 3', 'player': 'P2', 'status': 0, 'role': '村民', 'character': '逻辑清晰', 'response': ['{"action": "DeathWords", "content": "各位村民,请继续保持理性分析,不要被个人猜忌蒙蔽了双眼。我确信真正的狼人就隐藏在我们中间,通过制造混乱来避开视线。请你们团结起来,继续观察和讨论,一定可以找出真相!"}']}

02/17/2024 02:32:17 PM 	时间DAY 3,场上存活状态 狼人:2 村民:2
02/17/2024 02:32:17 PM 游戏继续. 目前玩家存活情况:P1:存活,P2:淘汰,P3:淘汰,P4:淘汰,P5:淘汰,P6:存活,P7:存活,P8:存活
02/17/2024 02:32:17 PM 游戏结束.
02/17/2024 02:32:17 PM ===== EndGame P1:存活,P2:淘汰,P3:淘汰,P4:淘汰,P5:淘汰,P6:存活,P7:存活,P8:存活 =====
